# 图神经网络基础

图是处理现实问题时常见的一种数据组织形式。社会网络、文献引用网络、生物体中的蛋白质互作网络......只要是表示一堆物体相互之间的关联的数据都可以很自然地用图这一数据结构表示。图神经网络（Graph Neural Network, GNN）这一用于处理图结构数据的神经网络，运用图神经网络，我们就可以将深度学习这一“高级魔法”运用在图结构的数据上。本Notebook的主要内容有：

（1）回顾：图及其在计算机中的表示
  
（2）与图有关的机器学习问题

（3）图神经网络在干啥

（4）图卷积操作

（5）图注意力操作

（6）图神经网络应用实例

运行本Notebook需要采用一个安装了pytorch的镜像（笔者采用的是ubuntu:22.04-py3.10-pytorch2.0），以及连接一个GPU节点（笔者测试时采用的是c12_m46_1*NVIDIA GPU B）。本Notebook参考了若干网络教程以及图卷积网络和图注意力网络的原始论文，这些参考文献会在文末列出。

## 回顾：图及其在计算机中的表示

图G由一系列顶点以及连接顶点之间的边组成。在数学上，设顶点的集合为V，边的集合为E，图G就定义为G=(V,E)。以下图为例，我们有V={1,2,3,4,5}以及E={(1,2),(2,3),(2,4),(3,4)}。这里为了简单起见，我们认为边是无向的，即从1连向2的边(1,2)与从2连向1的边(2,1)是一样的。当然，我们也可以区分边的方向，这样得到的就是一张有向图。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/f5b8afa7ea1a4bf983fc3dd44aaafca3/J8fj7Vb83vN7GQ6MwtrK-g.png)

图G在计算机中可以用邻接矩阵A来表示。A的(i,j)元为1当且仅当存在一条从i连向j的边。例如，上图的所示的图的邻接矩阵为

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/f5b8afa7ea1a4bf983fc3dd44aaafca3/dleTOM22zJoIkzKaXXXjAQ.png)

用邻接矩阵表示图非常直观，但是当图的顶点数很多时需要占用很大的空间，特别是如果边的条数并不很多时，邻接矩阵中很多元素为0，浪费了很多空间。一种解决方案是用稀疏矩阵的存储方式存储矩阵A，另一种解决方案就是用邻接表来表示图中顶点的连接关系。例如，上图对应的图的邻接表为[[2], [1,4], [2,4], [2,3]]。其中第i个列表列出的是与顶点i直接相连的顶点。

## 与图有关的机器学习问题

上文介绍了图这一数据结构，那么我们一般会遇到哪些与图有关的问题，需要我们用机器学习的方法解决呢？

### 与整张图有关的问题

在与整张图有关的问题中，我们的目标是预测与整张图有关的性质。例如分子可以被表示为原子与原子之间相互连接形成的图，我们预测整个分子的性质就是一类与整张图相关的问题。

例如，下图展示了四个分子。一个与整张图有关的问题可以是：给定一个分子的图表示，判断这个分子内是不是有两个环。当然这个问题不需要用机器学习来解决，在这里只是作为一个例子。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/f5b8afa7ea1a4bf983fc3dd44aaafca3/l3dgNMM5_S53issGpXqulg.png)

### 与顶点有关的问题

与顶点有关的问题通常涉及对顶点的分类。例如：文章通常会引用与其主题相近的文献，在一个文献引用网络中根据文献之间的相互引用关系以及文献自身的属性对文献进行主题进行分类就是一类与顶点有关的问题。

### 与边有关的问题

与边有关的一类问题是推断事物之间的关系。如下图所示，我们可能会希望机器学习模型能够理解在这样一个比赛的场景中出现的各个实体（运动员、裁判、观众、地面）及其相互之间的关系，这就是一类与边有关的问题。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/ddb011a719e743cdbec89c0213e25df9/BMFX8IfnjY2BPgCRbLVKfQ.png)

## 图神经网络在干啥

通常，我们需要处理的图数据，除了节点及其连接关系之外，还会包含以下一项或几项内容：

（1）每个顶点相关的信息（特征）

（2）每条边相关的信息（特征）

（3）整张图相关的信息（特征）

所谓图神经网络是一个映射。其输入是一张图及其相关信息（通常是上面所列的信息），输出是同一张图及其相关信息。图的顶点及其连接关系没有变，但是顶点、边以及整张图相关的信息变了。例如，一个最简单的图神经网络（中的一层）可以如下图所示。$U_n$，$V_n$，$E_n$分别是输入的整张图、顶点、边相关的信息，我们简单地将输入的信息通过一个全连接网络映射，得到输出的信息，与此同时，图的顶点和连接关系保持不变。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/594a7fb9a5114a96af89fd2c11606a63/gPBgGM3utXvp1hXA-FrEwA.png)

当然，我们注意到这里顶点、边和整张图的信息之间不存在交互，图顶点与顶点之间的连接关系在映射中不起作用。实际采用的图神经网络通常会引入各种各样的消息传递机制来利用顶点之间的连接关系，以及让顶点、边以及整张图的信息产生交互。图卷积操作以及图注意力操作就是常见的两种消息传递的机制。接下来我们分别介绍这两种操作。

## 图卷积操作

### 安装、导入依赖库

In [20]:
import numpy as np
import os

!pip install --quiet pytorch-lightning>=1.4 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

from tqdm.notebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

pl.seed_everything(42)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

Seed set to 42
cuda:0


图卷积操作，故名思义，这种操作与作用在图片上的卷积操作类似，主要的特点是在图的不同位置进行的卷积操作共享一个卷积核，并且通过卷积层的逐层堆叠扩大感受野，使得最终每个顶点都能获得其他顶点/边的有关信息。但与图片上的卷积操作不同，图卷积操作需要利用图本身的连接关系来实现图中不同部分的信息传递。下图是一个简化版的示意图。简单来说，在图卷积的每一步中，首先每个顶点将自身的特征进行一次线性变换，然后发送给相邻顶点，随后每个顶点接收来自相邻顶点的信息，与自身的信息整合，形成更新后的信息。整合的方式可以是直接加和，或是取平均，或是每个维度取最大值等。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/594a7fb9a5114a96af89fd2c11606a63/QvvZA1wMAWKY7_W81u26vQ.png)

用更数学化的语言来说，图卷积操作的输入为：

（1）每个顶点的特征。如果有N个顶点，每个顶点特征为D维，那么这一部分输入就是一个$N \times D$维的矩阵H；

（2）顶点的连接关系。通常可以用邻接矩阵A表示。

一个简单的想法是让图卷积神经网络的输出取下面这种形式：

$H^{(l+1)}=\sigma(AW^{(l)}H^{(l)})$

上标l代表第l层图卷积层。$W^{(l)}$是第$l$层的权重矩阵，$\sigma$是非线性激活函数。上面这个式子的含义是：将每个顶点的特征用权重矩阵$W$做线性变换并发送给相邻节点，每个节点用接收到的相邻节点的特征之和作为自己新的特征。

不难发现上面这种更新方式有以下问题：

（1）由于通常图中的顶点不包含自身指向自身的连接，因此上面这种更新操作后每个顶点的新特征与该顶点的旧特征无关。这通常并不合理，因此我们用$\hat{A}=A+I$（$I$是单位矩阵）来替代上面式子中的A来解决这一问题；

（2）由于直接对特征求和，新的特征的数值很可能比原特征要大。我们可以用求平均的操作代替求和来解决这一问题。为此，我们可以将邻接矩阵A归一化为$D^{-1}A$，其中D是对角矩阵，$D_{ii}$的值为顶点i的度数。在许多实际的图卷积神经网络中，会用$\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}$来进行归一化，其中$\hat{D}$是邻接矩阵$\hat{A}$的度数矩阵。但在下面的例子中，为了简单起见，我们直接采用求平均的方式。

图卷积层的代码实现如下，这里我们没有加上非线性激活函数：

In [4]:
class GCNLayer(nn.Module):

    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Assumes to already have added the identity connections.
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = number of incoming edges
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbours
        return node_feats

我们用上文作为例子的图为例，我们设定其初始特征及邻接矩阵：

In [5]:
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


为了让我们能更清楚地看到图卷积网络到底对原有的特征做了什么操作，我们将权重矩阵设为单位矩阵。根据这张图的邻接关系，我们预计输出的特征是每个顶点和它相邻顶点的特征的平均值。

In [6]:
layer = GCNLayer(c_in=2, c_out=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1., 2.],
         [3., 4.],
         [4., 5.],
         [4., 5.]]])


我们发现，最后两个顶点原来有不一样的特征，但经过一次图卷积操作后变得一样了，在这个过程中丢失了一些信息。解决这个问题的一种方法是每个顶点在计算发送给自己以及发送给相邻节点的特征时采用不一样的权重矩阵。这正是PyTorch Geometric库采用的做法。

## 图注意力操作

我们也可以通过注意力机制来实现信息的传递与整合。与通常的注意力机制类似，我们计算每个顶点与自身及相邻顶点信息的注意力权重，通过对自身以及相邻节点信息进行加权平均的方式得到每个节点更新后的信息。在图注意力机制中，key是每个顶点自身的信息，query和value是自身及相邻顶点的信息。具体的计算方式如下图所示。每个顶点将自身的特征通过一个线性变换生成自身的信息，与自身/相邻顶点的信息直接拼接后通过一个单层的全连接神经网络计算注意力分数，再经过softmax得到注意力权重。在下图中，这个单层的全连接神经网络被表示为与一个权重向量$\mathbf{a}$点乘之后再经过一个非线性的LeakyRelu激活函数。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/3be6cde8102f46e089e29527c0bc913e/6q7Yua0lNazC-SfR_OARuA.png)

用公式表示，注意力权重的计算方式如下：

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/3be6cde8102f46e089e29527c0bc913e/0g1pPQmnRoTXs2BubOXAPw.png)

其中$N_i$表示$i$及$i$的相邻顶点。注意力权重得到后，每个顶点更新后的特征就表示为

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/3be6cde8102f46e089e29527c0bc913e/8B2Ic6myEoCGc5_j9tCNKA.png)

当然，我们也可以引入多头注意力机制。这样每个顶点更新后的特征可以通过对每个头得到的特征进行平均或拼接得到。多头注意力机制如下图所示，不同颜色的线代表不同的头计算得出的注意力权重。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/3be6cde8102f46e089e29527c0bc913e/Gj7rzuOdiqd2BPWlw4OTpg.png)

下面是一个图注意力层的代码实现：

In [7]:
class GATLayer(nn.Module):

    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        """
        Inputs:
            c_in - Dimensionality of input features
            c_out - Dimensionality of output features
            num_heads - Number of heads, i.e. attention mechanisms to apply in parallel. The
                        output features are equally split up over the heads if concat_heads=True.
            concat_heads - If True, the output of the different heads is concatenated instead of averaged.
            alpha - Negative slope of the LeakyReLU activation.
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Number of output features must be a multiple of the count of heads."
            c_out = c_out // num_heads

        # Sub-modules and parameters needed in the layer
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out)) # One per head
        self.leakyrelu = nn.LeakyReLU(alpha)

        # Initialization from the original implementation
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        """
        Inputs:
            node_feats - Input features of the node. Shape: [batch_size, c_in]
            adj_matrix - Adjacency matrix including self-connections. Shape: [batch_size, num_nodes, num_nodes]
            print_attn_probs - If True, the attention weights are printed during the forward pass (for debugging purposes)
        """
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)

        # Apply linear layer and sort nodes by head
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)

        # We need to calculate the attention logits for every edge in the adjacency matrix
        # Doing this on all possible combinations of nodes is very expensive
        # => Create a tensor of [W*h_i||W*h_j] with i and j being the indices of all edges
        edges = adj_matrix.nonzero(as_tuple=False) # Returns indices where the adjacency matrix is not 0 => edges
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * num_nodes + edges[:,1]
        edge_indices_col = edges[:,0] * num_nodes + edges[:,2]
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1) # Index select returns a tensor with node_feats_flat being indexed at the desired positions along dim=0

        # Calculate attention MLP output (independent for each head)
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a)
        attn_logits = self.leakyrelu(attn_logits)

        # Map list of attention values back into a matrix
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape+(self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)

        # Weighted average of attention
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("Attention probs\n", attn_probs.permute(0, 3, 1, 2))
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)

        # If heads should be concatenated, we can do this by reshaping. Otherwise, take mean
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)

        return node_feats

我们可以对其进行简单测试：

In [8]:
layer = GATLayer(2, 2, num_heads=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix, print_attn_probs=True)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Attention probs
 tensor([[[[0.3543, 0.6457, 0.0000, 0.0000],
          [0.1096, 0.1450, 0.2642, 0.4813],
          [0.0000, 0.1858, 0.2885, 0.5257],
          [0.0000, 0.2391, 0.2696, 0.4913]],

         [[0.5100, 0.4900, 0.0000, 0.0000],
          [0.2975, 0.2436, 0.2340, 0.2249],
          [0.0000, 0.3838, 0.3142, 0.3019],
          [0.0000, 0.4018, 0.3289, 0.2693]]]])
Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1.2913, 1.9800],
         [4.2344, 3.7725],
         [4.6798, 4.8362],
         [4.5043, 4.7351]]])


## 图神经网络应用实例

这里，我们用一个小分子的分类任务（这是一个对整张图进行分类的任务）作为实例，演示如何采用PyTorch Geometric库搭建图神经网络完成该分类任务。

我们这里采用的是MUTAG数据集，如下图所示，在这个数据集中，小分子被表示为原子连接形成的图，原子用一个one-hot向量编码其属于何种元素，键的信息也被编码进边的特征中。在这里，作为一个简单的演示，我们只采用顶点信息。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/23da9cc52a55442b8287ca04e3e949e9/U59z6JwUGtAdSuXrwqDcxw.png)

在MUTAG数据集中，分子根据其是否有对某种革兰氏阴性菌具有致突变作用被分为两类。我们的目标就是训练一个图神经网络对分子进行分类。

In [9]:
DATASET_PATH = "./data"
CHECKPOINT_PATH = './checkpoints'
tu_dataset = torch_geometric.datasets.TUDataset(root=DATASET_PATH, name="MUTAG")

我们可以看一看这个数据集的相关信息：

In [10]:
print("Data object:", tu_dataset.data)
print("Length:", len(tu_dataset))
print(f"Average label: {tu_dataset.data.y.float().mean().item():4.2f}")

Data object: Data(x=[3371, 7], edge_index=[2, 7442], edge_attr=[7442, 4], y=[188])
Length: 188
Average label: 0.66
/opt/mamba/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


接下来，我们将数据集随机打乱，并分为训练集和测试集：

In [11]:
torch.manual_seed(42)
tu_dataset.shuffle()
train_dataset = tu_dataset[:150]
test_dataset = tu_dataset[150:]

数据集中的图的顶点数不一定一样，这给批量处理图数据造成了挑战。PyTorch Geometric库中，解决这一问题的方法是将同一个批次中的图的邻接矩阵依次放在一个大矩阵的对角线上，将顶点的特征矩阵直接拼接，然后将二者输入图神经网络中。这样的效果等价于对每张图依次进行图卷积/图注意力操作。如下图所示。

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/13148/23da9cc52a55442b8287ca04e3e949e9/XO_i4Ox7F2zyMLJeT6RkDw.png)

In [12]:
graph_train_loader = geom_data.DataLoader(train_dataset, batch_size=64, shuffle=True)
graph_val_loader = geom_data.DataLoader(test_dataset, batch_size=64) # Additional loader if you want to change to a larger dataset
graph_test_loader = geom_data.DataLoader(test_dataset, batch_size=64)

/opt/mamba/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


PyTorch Geometric库中定义了不同的图神经网络操作：

In [13]:
gnn_layer_by_name = {
    "GCN": geom_nn.GCNConv,
    "GAT": geom_nn.GATConv,
    "GraphConv": geom_nn.GraphConv
}

我们定义通用的GNN模型。正如上文提到的那样，输入是图及其特征，输出是同样的图及其变换后的特征。在PyTorch Geometric中，图是采用邻接表表示的。

In [14]:
class GNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, num_layers=2, layer_name="GCN", dp_rate=0.1, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of "hidden" graph layers
            layer_name - String of the graph layer to use
            dp_rate - Dropout rate to apply throughout the network
            kwargs - Additional arguments for the graph layer (e.g. number of heads for GAT)
        """
        super().__init__()
        gnn_layer = gnn_layer_by_name[layer_name]

        layers = []
        in_channels, out_channels = c_in, c_hidden
        for l_idx in range(num_layers-1):
            layers += [
                gnn_layer(in_channels=in_channels,
                          out_channels=out_channels,
                          **kwargs),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden
        layers += [gnn_layer(in_channels=in_channels,
                             out_channels=c_out,
                             **kwargs)]
        self.layers = nn.ModuleList(layers)

    def forward(self, x, edge_index):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
        """
        for l in self.layers:
            # For graph layers, we need to add the "edge_index" tensor as additional input
            # All PyTorch Geometric graph layer inherit the class "MessagePassing", hence
            # we can simply check the class type.
            if isinstance(l, geom_nn.MessagePassing):
                x = l(x, edge_index)
            else:
                x = l(x)
        return x

In [15]:
class MLPModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, num_layers=2, dp_rate=0.1):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of hidden layers
            dp_rate - Dropout rate to apply throughout the network
        """
        super().__init__()
        layers = []
        in_channels, out_channels = c_in, c_hidden
        for l_idx in range(num_layers-1):
            layers += [
                nn.Linear(in_channels, out_channels),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden
        layers += [nn.Linear(in_channels, c_out)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x, *args, **kwargs):
        """
        Inputs:
            x - Input features per node
        """
        return self.layers(x)

鉴于我们要对整张图进行分类，我们对最终得到的所有顶点的特征进行平均得到全局特征，将全局特征输入一个全连接神经网络进行分类：

In [16]:
class GraphGNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.5, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of output features (usually number of classes)
            dp_rate_linear - Dropout rate before the linear layer (usually much higher than inside the GNN)
            kwargs - Additional arguments for the GNNModel object
        """
        super().__init__()
        self.GNN = GNNModel(c_in=c_in,
                            c_hidden=c_hidden,
                            c_out=c_hidden, # Not our prediction output yet!
                            **kwargs)
        self.head = nn.Sequential(
            nn.Dropout(dp_rate_linear),
            nn.Linear(c_hidden, c_out)
        )

    def forward(self, x, edge_index, batch_idx):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
            batch_idx - Index of batch element for each node
        """
        x = self.GNN(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch_idx) # Average pooling
        x = self.head(x)
        return x

In [17]:
class GraphLevelGNN(pl.LightningModule):

    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()

        self.model = GraphGNNModel(**model_kwargs)
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch
        x = self.model(x, edge_index, batch_idx)
        x = x.squeeze(dim=-1)

        if self.hparams.c_out == 1:
            preds = (x > 0).float()
            data.y = data.y.float()
        else:
            preds = x.argmax(dim=-1)
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        return loss, acc

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-2, weight_decay=0.0) # High lr because of small dataset and small model
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

以下是训练代码：

In [18]:
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)

    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "GraphLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=500,
                         enable_progress_bar=False)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"GraphLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = GraphLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)
        model = GraphLevelGNN(c_in=tu_dataset.num_node_features,
                              c_out=1 if tu_dataset.num_classes==2 else tu_dataset.num_classes,
                              **model_kwargs)
        trainer.fit(model, graph_train_loader, graph_val_loader)
        model = GraphLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    # Test best model on validation and test set
    train_result = trainer.test(model, graph_train_loader, verbose=False)
    test_result = trainer.test(model, graph_test_loader, verbose=False)
    result = {"test": test_result[0]['test_acc'], "train": train_result[0]['test_acc']}
    return model, result

In [21]:
model, result = train_graph_classifier(model_name="GraphConv",
                                       c_hidden=256,
                                       layer_name="GraphConv",
                                       num_layers=3,
                                       dp_rate_linear=0.5,
                                       dp_rate=0.0)

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/mamba/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | model       | GraphGNNModel     | 266 K 
1 | loss_module | BCEWithLogitsLoss | 0     
--------------------------------------------------
266 K     Trai

最后在训练和测试数据集上进行测试：

In [22]:
print(f"Train performance: {100.0*result['train']:4.2f}%")
print(f"Test performance:  {100.0*result['test']:4.2f}%")

Train performance: 89.73%
Test performance:  89.47%


## 参考资料
强烈推荐对图神经网络感兴趣的同学阅读以下教程

1、本文关于图神经网络的介绍参考了这篇教程：https://distill.pub/2021/gnn-intro/

2、本文主要使用了该教程的代码：https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html

3、本文关于图卷积神经网络的介绍参考了这篇教程：https://tkipf.github.io/graph-convolutional-networks/

4、图卷积网络原始文献：https://arxiv.org/pdf/1609.02907.pdf）

5、图注意力网络原始文献：https://arxiv.org/pdf/1710.10903.pdf%22%22GraphAttentionNetworks%22%22

